In [2]:
import sys
import os
sys.path.append('./src/occ_net')
sys.path.append('./src/')

We then import all the necessary packages and library functions

In [3]:
import os.path as osp
import torch
import numpy as np
import trimesh
import random
import copy
from scipy.spatial.transform import Rotation
from imageio import get_writer, imread

from ndf_robot.utils import path_util, torch_util, trimesh_util
import ndf_robot.model.vnn_occupancy_net_pointnet_dgcnn as vnn_occupancy_network
from ndf_robot.eval.ndf_alignment import NDFAlignmentCheck

## Check occupancy network meshes

In [4]:
from torch.utils.data import DataLoader
from ndf_robot.training.acid_dataset import AcidDataset
root_dir = "/home/sjindal/work/ACID/data_ndf"
category = ["teddy"]

train_dataset = AcidDataset(root_dir, "train", category, transform=True, all_points=False)
val_dataset = AcidDataset(root_dir, "val", category, transform=False, all_points=False)

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True,
                              drop_last=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True,
                            drop_last=True, num_workers=4)

In [5]:
model = vnn_occupancy_network.VNNOccNet(latent_dim=256, model_type='pointnet', return_features=True, sigmoid=True).cuda()
model_path = "logs/ndf_training_exp/checkpoints/model_current.pth"
model.load_state_dict(torch.load(model_path))
model.eval();

In [6]:
model_input, gt_occ = next(iter(train_dataloader))
model_input["point_cloud"] = model_input["point_cloud"].cuda()
model_input["coords"] = model_input["coords"].cuda()
gt_occ["occ"] = gt_occ["occ"].cuda()

## Create a mesh

In [10]:
import torch
# import torch.distributions as dist
import os
import shutil
import argparse
from tqdm import tqdm
import time
from collections import defaultdict
import pandas as pd
from im2mesh import config
from im2mesh.checkpoints import CheckpointIO
from im2mesh.utils.io import export_pointcloud
from im2mesh.utils.visualize import visualize_data
from im2mesh.utils.voxels import VoxelGrid

./src/occ_net/im2mesh/utils/binvox_rw.py:214: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  def sparse_to_dense(voxel_data, dims, dtype=np.bool):


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations